In [1]:
import numpy as np
from qiskit import IBMQ, Aer, assemble, transpile, execute
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit.providers.ibmq import least_busy
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.algorithms import NumPyEigensolver
from numpy import linalg as LA
from scipy.linalg import expm, norm

In [2]:
def is_unitary(m):
    return norm(np.eye(len(m)) - m.dot(m.T.conj()))
def is_hermitian(m):
    return norm(m-m.T.conj())

In [3]:
coefficients = []
bases = []
with open('LiH-Hamiltonian.txt') as f:
    for line in f:
        x = line.strip().split(" ")
        if (len(line) > 1):
            c = float(x[1])
            c = (c if x[0] == '+' else -c)
            coefficients.append(c)
            bases.append(x[3])

# comment this out when stuff actually works
# bases = ['XXY', 'ZXY', 'YYY', 'IYZ'] # these are all commuting
# coefficients = [1]*len(bases)
hamiltonian = np.zeros((2**len(bases[0]), 2**len(bases[0])), dtype=np.cdouble)
for c, lbl in zip(coefficients, bases):
    op = Operator(Pauli(label=lbl))
    hamiltonian+=c*op.data

In [4]:
hamiltonian

array([[-4.99600361e-16+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
        ...,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
         0.00000000e+00+0.j],
       [ 0.00000000e+00+0.j, -7.87844742e-01+0.j,  4.69344975e-02+0.j,
        ...,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
         0.00000000e+00+0.j],
       [ 0.00000000e+00+0.j,  4.69344975e-02+0.j, -3.62117484e-01+0.j,
        ...,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
         0.00000000e+00+0.j],
       ...,
       [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
        ...,  6.54696622e+00+0.j, -6.66047765e-02+0.j,
         0.00000000e+00+0.j],
       [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
        ..., -6.66047765e-02+0.j,  6.47684874e+00+0.j,
         0.00000000e+00+0.j],
       [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
        ...,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
         8.46875477e+00+0.j]])

In [5]:
is_hermitian(hamiltonian)

0.0

In [6]:
def get_circuit(t=1, n=1):
    qc = QuantumCircuit(len(bases[0]))
    for i in range(n):
        for c, b in zip(coefficients, bases):
            # implement e^{-i*c*b*t/n}
            b = b[::-1]
            q = []
            qc.barrier()
            for i, op in enumerate(b):
                if (op == 'X'):
                    qc.h(i)
                    q.append(i)
                elif (op == 'Z'):
                    q.append(i)
                elif (op == 'Y'):
                    qc.sdg(i)
                    qc.h(i)
                    q.append(i)
                elif (op == 'I'):
                    continue
            qc.barrier()
            if (len(q) > 0):
                if (len(q) > 1):
                    for i in range(len(q)-1):
                        qc.cx(q[i], q[-1])
                qc.rz(2*c*t/n, q[-1])
                if (len(q) > 1):
                    for i in reversed(range(len(q)-1)):
                        qc.cx(q[i], q[-1])
                qc.barrier()
                for i, op in enumerate(b):
                    if (op == 'X'):
                        qc.h(i)
                    elif (op == 'Z'):
                        continue
                    elif (op == 'Y'):
                        qc.h(i)
                        qc.s(i)
                    elif (op == 'I'):
                        continue
            qc.barrier()
    return qc

def get_unitaries(t=1, n=1):
    e_ith = expm(-1.j * t * hamiltonian)
    qc = get_circuit(t=t, n=n)
    backend = Aer.get_backend('unitary_simulator')
    job = execute(qc, backend)
    result = job.result()
    U_circuit = result.get_unitary(qc)
    return e_ith, U_circuit

In [7]:
get_circuit(n=1).draw()

░ ┌─────┐┌───┐ ░                                                 ░ ┌───┐»
q_0: ─░─┤ Sdg ├┤ H ├─░───■─────────────────────────────────────────■───░─┤ H ├»
      ░ ├─────┤├───┤ ░   │                                         │   ░ ├───┤»
q_1: ─░─┤ Sdg ├┤ H ├─░───┼────■───────────────────────────────■────┼───░─┤ H ├»
      ░ └─────┘└───┘ ░   │    │                               │    │   ░ └───┘»
q_2: ─░──────────────░───┼────┼───────────────────────────────┼────┼───░──────»
      ░              ░   │    │                               │    │   ░      »
q_3: ─░──────────────░───┼────┼───────────────────────────────┼────┼───░──────»
      ░              ░   │    │                               │    │   ░      »
q_4: ─░──────────────░───┼────┼───────────────────────────────┼────┼───░──────»
      ░ ┌─────┐┌───┐ ░   │    │                               │    │   ░ ┌───┐»
q_5: ─░─┤ Sdg ├┤ H ├─░───┼────┼────■─────────────────────■────┼────┼───░─┤ H ├»
      ░ ├─────┤├───┤ ░ ┌─┴─┐┌─┴─┐┌─┴─┐┌───────────────┐┌─┴─┐┌─┴─┐┌─┴─┐ ░ ├───┤»
q_6: ─░─┤ Sdg ├┤ H ├─░─┤ X ├┤ X ├┤ X ├┤ Rz(0.0060693) ├┤ X ├┤ X ├┤ X ├─░─┤ H ├»
      ░ └─────┘└───┘ ░ └───┘└───┘└───┘└───────────────┘└───┘└───┘└───┘ ░ └───┘»
q_7: ─░──────────────░─────────────────────────────────────────────────░──────»
      ░              ░                                                 ░      »
q_8: ─░──────────────░─────────────────────────────────────────────────░──────»
      ░              ░                                                 ░      »
q_9: ─░──────────────░─────────────────────────────────────────────────░──────»
      ░              ░                                                 ░      »
«     ┌───┐ ░  ░ ┌─────┐┌───┐ ░                                                »
«q_0: ┤ S ├─░──░─┤ Sdg ├┤ H ├─░───■─────────────────────────────────────────■──»
«     ├───┤ ░  ░ ├─────┤├───┤ ░   │                                         │  »
«q_1: ┤ S ├─░──░─┤ Sdg ├┤ H ├─░───┼────■───────────────────────────────■────┼──»
«     └───┘ ░  ░ └─────┘└───┘ ░   │    │                               │    │  »
«q_2: ──────░──░──────────────░───┼────┼───────────────────────────────┼────┼──»
«           ░  ░              ░   │    │                               │    │  »
«q_3: ──────░──░──────────────░───┼────┼───────────────────────────────┼────┼──»
«           ░  ░              ░   │    │                               │    │  »
«q_4: ──────░──░──────────────░───┼────┼───────────────────────────────┼────┼──»
«     ┌───┐ ░  ░  ┌───┐       ░   │    │                               │    │  »
«q_5: ┤ S ├─░──░──┤ H ├───────░───┼────┼────■─────────────────────■────┼────┼──»
«     ├───┤ ░  ░  ├───┤       ░ ┌─┴─┐┌─┴─┐┌─┴─┐┌───────────────┐┌─┴─┐┌─┴─┐┌─┴─┐»
«q_6: ┤ S ├─░──░──┤ H ├───────░─┤ X ├┤ X ├┤ X ├┤ Rz(0.0060693) ├┤ X ├┤ X ├┤ X ├»
«     └───┘ ░  ░  └───┘       ░ └───┘└───┘└───┘└───────────────┘└───┘└───┘└───┘»
«q_7: ──────░──░──────────────░────────────────────────────────────────────────»
«           ░  ░              ░                                                »
«q_8: ──────░──░──────────────░────────────────────────────────────────────────»
«           ░  ░              ░                                                »
«q_9: ──────░──░──────────────░────────────────────────────────────────────────»
«           ░  ░              ░                                                »
«      ░ ┌───┐┌───┐ ░  ░  ┌───┐       ░                                      »
«q_0: ─░─┤ H ├┤ S ├─░──░──┤ H ├───────░───■──────────────────────────────────»
«      ░ ├───┤├───┤ ░  ░  ├───┤       ░   │                                  »
«q_1: ─░─┤ H ├┤ S ├─░──░──┤ H ├───────░───┼────■─────────────────────────────»
«      ░ └───┘└───┘ ░  ░  └───┘       ░   │    │                             »
«q_2: ─░────────────░──░──────────────░───┼────┼─────────────────────────────»
«      ░            ░  ░              ░   │    │                             »
«q_3: ─░────────────░──░──────────────░───┼────┼────────────────────────

In [ ]:
e_ith, U_circuit = get_unitaries(n=1)

In [9]:
def get_error(e_ith, U_circuit):
    diff = U_circuit.data - e_ith
    # measure the operator's accuracy
    w, v = LA.eig(diff)

    return np.absolute(max(w))

get_error(e_ith, U_circuit)

1.0229505671952006

In [10]:
np.around(e_ith, decimals=3)

array([[ 1.   +0.j   ,  0.   +0.j   ,  0.   +0.j   , ...,  0.   +0.j   ,
         0.   +0.j   ,  0.   +0.j   ],
       [ 0.   +0.j   ,  0.697+0.703j,  0.029-0.037j, ...,  0.   +0.j   ,
         0.   +0.j   ,  0.   +0.j   ],
       [ 0.   +0.j   ,  0.029-0.037j,  0.932+0.353j, ...,  0.   +0.j   ,
         0.   +0.j   ,  0.   +0.j   ],
       ...,
       [ 0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   , ...,  0.963-0.26j ,
         0.016+0.065j,  0.   +0.j   ],
       [ 0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   , ...,  0.016+0.065j,
         0.973-0.192j,  0.   +0.j   ],
       [ 0.   +0.j   ,  0.   +0.j   ,  0.   +0.j   , ...,  0.   +0.j   ,
         0.   +0.j   , -0.577-0.817j]])

In [11]:
np.around(U_circuit, decimals=3)

array([[ 0.479+0.878j, -0.   -0.j   , -0.   -0.j   , ...,  0.   +0.j   ,
        -0.   -0.j   ,  0.   +0.j   ],
       [-0.   +0.j   , -0.281+0.949j,  0.045+0.013j, ...,  0.   -0.j   ,
        -0.   -0.j   ,  0.   -0.j   ],
       [-0.   +0.j   ,  0.048+0.002j,  0.137+0.987j, ...,  0.   -0.j   ,
         0.   +0.j   , -0.   -0.j   ],
       ...,
       [-0.   -0.j   ,  0.   -0.j   ,  0.   +0.j   , ...,  0.69 +0.721j,
        -0.046+0.048j, -0.   +0.j   ],
       [ 0.   -0.j   , -0.   +0.j   , -0.   -0.j   , ..., -0.051+0.042j,
         0.634+0.763j, -0.   -0.j   ],
       [ 0.   -0.j   ,  0.   +0.j   ,  0.   -0.j   , ...,  0.   -0.j   ,
         0.   -0.j   ,  0.44 -0.898j]])

In [12]:
hamiltonian

array([[-4.99600361e-16+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
        ...,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
         0.00000000e+00+0.j],
       [ 0.00000000e+00+0.j, -7.87844742e-01+0.j,  4.69344975e-02+0.j,
        ...,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
         0.00000000e+00+0.j],
       [ 0.00000000e+00+0.j,  4.69344975e-02+0.j, -3.62117484e-01+0.j,
        ...,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
         0.00000000e+00+0.j],
       ...,
       [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
        ...,  6.54696622e+00+0.j, -6.66047765e-02+0.j,
         0.00000000e+00+0.j],
       [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
        ..., -6.66047765e-02+0.j,  6.47684874e+00+0.j,
         0.00000000e+00+0.j],
       [ 0.00000000e+00+0.j,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
        ...,  0.00000000e+00+0.j,  0.00000000e+00+0.j,
         8.46875477e+00+0.j]])

In [13]:
e_ith-U_circuit.data

array([[ 5.20689551e-01-8.77645426e-01j,  3.51493888e-15+1.55042752e-15j,
         3.68805396e-15+5.02537662e-16j, ...,
        -7.62125350e-48-4.24024420e-47j,  1.55470562e-47+6.87552408e-48j,
        -5.13225950e-48-1.67990854e-48j],
       [ 3.01547881e-15-5.80557878e-16j,  9.78650028e-01-2.46215921e-01j,
        -1.53347572e-02-5.01654790e-02j, ...,
        -1.67239973e-47+1.52780772e-47j,  8.22702617e-48+3.60478725e-48j,
        -1.29569180e-48+2.31727708e-47j],
       [ 1.99407492e-15-3.24781196e-15j, -1.85707972e-02-3.93918853e-02j,
         7.94874765e-01-6.34201105e-01j, ...,
        -5.75971546e-48+5.26466817e-48j, -1.84846497e-47-1.30975759e-47j,
         2.09822747e-47+1.30449681e-48j],
       ...,
       [ 1.55912942e-47+1.42357412e-48j, -8.08317770e-47+2.78981603e-47j,
        -5.20148031e-48-3.36342988e-48j, ...,
         2.73399233e-01-9.80832120e-01j,  6.21475933e-02+1.68386658e-02j,
         3.12438884e-15-4.46021860e-16j],
       [-3.08229614e-48+1.78943150e-47j,  9.

10 qubits -> 2^10 states -> 2^20 entries ($\approx$ 1,000,000, which is quite reasonable)

# New Stuff

In [14]:
def buildPauliString(qc, pauliString, t, n):
    c, b = pauliString
    # implement e^{-i*c*b*t/n}
    b = b[::-1]
    q = []
    qc.barrier()
    for i, op in enumerate(b):
        if (op == 'X'):
            qc.h(i)
            q.append(i)
        elif (op == 'Z'):
            q.append(i)
        elif (op == 'Y'):
            qc.sdg(i)
            qc.h(i)
            q.append(i)
        elif (op == 'I'):
            continue
    qc.barrier()
    if (len(q) > 0):
        if (len(q) > 1):
            for i in range(len(q)-1):
                qc.cx(q[i], q[-1])
        qc.rz(2*c*t/n, q[-1])
        if (len(q) > 1):
            for i in reversed(range(len(q)-1)):
                qc.cx(q[i], q[-1])
        qc.barrier()
        for i, op in enumerate(b):
            if (op == 'X'):
                qc.h(i)
            elif (op == 'Z'):
                continue
            elif (op == 'Y'):
                qc.h(i)
                qc.s(i)
            elif (op == 'I'):
                continue
    qc.barrier()
    return qc

def buildHamiltonian(qc, pauliStrings, t, n):
    for pauliString in pauliStrings:
        # print(pauliString[1])
        qc = buildPauliString(qc, pauliString, t, n)
    return qc

In [15]:
def testMethods(t=1, n=1):
    qc = QuantumCircuit(len(bases[0]))
    pauliStrings = list(zip(coefficients, bases))
    for i in range(n):
        qc = buildHamiltonian(qc, pauliStrings, t, n)
        
    e_ith = expm(-1.j * t * hamiltonian)
    backend = Aer.get_backend('unitary_simulator')
    job = execute(qc, backend)
    result = job.result()
    U_circuit = result.get_unitary(qc)
    return get_error(e_ith, U_circuit)

# first order trotter formulation.
def get_circuit_1(t=1, n=1):
    qc = QuantumCircuit(len(bases[0]))
    pauliStrings = list(zip(coefficients, bases))
    for i in range(n):
        qc = buildHamiltonian(qc, pauliStrings, t, n)
    return qc

# second order trotter formulation.
def get_circuit_2(t=1, n=1):
    qc = QuantumCircuit(len(bases[0]))
    pauliStrings = list(zip(coefficients, bases))
    reversedPauliStrings = list(zip(reversed(coefficients), reversed(bases)))
    
    for i in range(n):
        qc = buildHamiltonian(qc, pauliStrings, t, n * 2)
        qc = buildHamiltonian(qc, reversedPauliStrings, t, n * 2)
        # qc = buildHamiltonian(qc, pauliStrings, t, n)
    return qc

# kth order trotter formulation.
def get_circuit_k(t=1, n=1, k=1):
    if k == 1:
        return get_circuit_1(t,n)
    if k == 2:
        return get_circuit_2(t,n)
    elif (k % 2 == 0):
        s_k = (4 - 4**(1 / (k - 1)))**(-1)
        c1 = get_circuit_k(s_k * t, n, k-2)
        c1_2 = c1.compose(c1)
        c2 = get_circuit_k((1 - 4 * s_k) * t, n, k-2)
        c = c1_2.compose(c2).compose(c1_2)
        return c
    else:
        return "make k even!!"

def get_unitaries_k(t=1, n=1, k=1):
    e_ith = expm(-1.j * t * hamiltonian)
    qc = get_circuit_k(t=t, n=n, k=k)
    backend = Aer.get_backend('unitary_simulator')
    job = execute(qc, backend)
    result = job.result()
    U_circuit = result.get_unitary(qc)
    return e_ith, U_circuit

In [16]:
# testMethods(n=100)

In [17]:
# e_ith2, U_circuit2 = get_unitaries_2(n=40)

In [18]:
# get_error(e_ith2, U_circuit2)

In [ ]:
e_ith4, U_circuit4 = get_unitaries_k(n=40, k=4)

In [ ]:
get_error(e_ith4, U_circuit4)

In [ ]:
np.around(e_ith2, decimals=3)

In [ ]:
np.around(U_circuit2, decimals=3)